In [1]:
from llm_study import *

In [2]:
from langchain.chains.summarize import load_summarize_chain
from langchain.llms.openai import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

### Load globally used LLMs and chains

In [3]:
doc_embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

react_llm = OpenAI(model="text-davinci-003", temperature=0.5)

summarize_llm = OpenAI(model="text-davinci-003", temperature=0.0)

knowledge_summarizer = KnowledgeSummarizer(llm=summarize_llm)

scene_summarizer = SceneSummarizer(llm=summarize_llm)

def vectorstore_factory(*docs: Document):
    return FAISS.from_documents(docs or [KnowledgeDocTypes.blank()], doc_embeddings)

### Define ReAct examples

In [4]:
react_examples = [
    """
    Summary: You are shopping at the supermarket. A stranger approached you.
    Obs: The stranger says: So, do you like soccer?
    Act: [Think] I must recall if I like soccer
    Obs: OK
    Act: [Recall] Do I like soccer?
    Obs: Played soccer with the coworkers last sunday afternoon
    Act: [Say] Yes, I play it often with my coworkers
    """,
    """
    Summary: A man named Mark Lewis approached you making small talk about the bad weather
    Obs: Mark Lewis says: So, are you married?
    Act: [Think] I must recall if I am married
    Obs: OK
    Act: [Recall] Am I married?
    Obs: John black married Henrietta Black in 1892
    Act: [Say] Yes, since 1892. Why do you ask?
    """,
    """
    Summary: You are in the university. You are studying for a test
    Obs: Your mother says: What are you studying?
    Act: [Think] I must recall what I am studying
    Obs: OK
    Act: [Recall] What test am I studying for
    Obs: statistics test next week
    Act: [Say] I am studying for a statistics test
    """,
    # """
    # Summary: You are concentrated in reading a book. A child approached you
    # Obs: The child did the following: stare at you
    # Act: [Think] I'm too concentrated, maybe I'll ignore it
    # Obs: OK
    # Act: [Do] ignore the child
    # """,
    # """
    # Summary: You are concentrated in reading a book. You overhear coworkers chatting about soccer
    # Obs: A coworker said: I'm betting they'll win 2-0 this time
    # Act: [Think] I must recall if I have interest in soccer
    # Obs: OK
    # Act: [Recall] my interest in soccer
    # Obs: No information was found
    # Act: [Think] I have no interest in this, I'd better ignore
    # Obs: OK
    # Act: [Do] keep reading the book while ignoring the discussion
    # """,
    # """
    # Summary: You went to a book store and asked the steward for the book Frankenstren from Mary Shelley.
    # Obs: The steward said: Wait a moment, I'm checking if we have it in our stock
    # Act: [Think] I must wait until they check it
    # Obs: OK
    # Act: [Do] wait patiently
    # """,
]

example_selector = ReActUtils.create_similarity_examples_selector(
    examples=[ReActExample(content=e) for e in react_examples],
    embeddings=doc_embeddings,
    vectorstore_cls=FAISS,
    num_examples=1,
)

### Define the NpcSheet model

In [5]:
TMPL_NPC_SUMMARY = textwrap.dedent("""
    Your name is {name}
    You are {age} years old
    Your personality is {personality_traits}
    About you: {description}
""").strip()

class NpcSheet(BaseNpcSheet):
    name: str
    age: int
    personality_traits: List[str]
    description: str

    @property
    def summary(self) -> str:
        kwargs = self.__dict__.copy()
        kwargs["personality_traits"] = ", ".join(kwargs["personality_traits"])
        return TMPL_NPC_SUMMARY.format(**kwargs)

### Declare NPCs

In [6]:
john = Npc(
    sheet=NpcSheet(
        name="John Bennet",
        age=24,
        personality_traits=["polite", "gentle", "humble"],
        description="You are a lawyer",
    ),
    knowledge_base=NpcKnowledge(
        max_retrieval=2,
        min_relevance=0.7,
        summarizer=knowledge_summarizer,
        vectorstore=vectorstore_factory(
            KnowledgeDocTypes.scene("""
                I was enjoying a drink in a London pub when I was approached by a stranger, Buck Jones,
                who introduced himself as a dietary doctor. After I told him I did not have a sister,
                he apologized for not introducing himself sooner and we exchanged pleasantries. He then
                gave me some advice about nutrition and health, emphasizing the importance of avoiding
                sugar and carbs. I told him I was familiar with the basics of nutrition and health and
                that I try to maintain a balanced diet
            """),
        ),
    ),
)

becky = Npc(
    sheet=NpcSheet(
        name="Becky Jameson",
        age=31,
        personality_traits=["bold", "outgoing", "inquisitive"],
        description="You are a detective",
    ),
    knowledge_base=NpcKnowledge(
        max_retrieval=2,
        min_relevance=0.7,
        summarizer=knowledge_summarizer,
        vectorstore=vectorstore_factory(
            KnowledgeDocTypes.scene("""
                Got a new case to work: Some creep has been asking people
                around town if they have a sister. Yesterday a body of a
                girl was found in the river. There might be a connection.
            """),
        ),
    ),
)

### Create Scenes

In [7]:
def create_scene(npc: Npc, situation: str, trigger: str) -> NpcScene:
    npc.situation = situation
    agent = NpcAgent(
        npc=npc,
        llm=react_llm,
        example_selector=example_selector,
        role_play_tools=[SpeechTool.create()],
    )
    return NpcScene(agent, scene_summarizer, trigger=trigger)

In [8]:
scene_john = create_scene(
    john,
    situation="You are relaxing in a cafe in Berlin",
    trigger=f"{becky.sheet.name} approached you"
)

scene_becky = create_scene(
    becky,
    situation="You are in a cafe in Berlin. You are investigating a crime",
    trigger=f"You approached {john.sheet.name}"
)

In [9]:
def run_two_way_scene(active: NpcScene, passive: NpcScene, first_action: RolePlayAction, turns: int):
    active.add_to_summary(first_action.as_own())
    actions = [first_action]
    for _ in range(turns):
        actions.append(passive.interact(actions[-1]))
        actions.append(active.interact(actions[-1]))
    return actions

In [10]:
actions = run_two_way_scene(
    scene_becky,
    scene_john,
    SpeechAction(actor=becky.sheet.name, text="Hi, do you have time to talk?"),
    turns=3,
)



> Entering new AgentExecutor chain...

[Think] I should be polite and respond to her.
OK
[Say] Hi Becky, I have a few minutes. What would you like to talk about?


> Finished chain.


> Entering new AgentExecutor chain...

[Say] Hi John, I'm looking into a case and I was wondering if you could provide me with any information that could help me?


> Finished chain.


> Entering new AgentExecutor chain...

[Think] I should recall what information I have that could help her.
OK
[Recall] What information do I have that could help her?


/home/lariel/workspace/llm-study/.venv/lib/python3.9/site-packages/langchain/vectorstores/base.py:269: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.7
  warnings.warn(


no information was found
[Say] Unfortunately, I don't have any information that could help you.


> Finished chain.


> Entering new AgentExecutor chain...

[Think] What other questions can I ask him?
OK
[Recall] What other people have I spoken to about this case?
I have been assigned a new case involving a suspicious individual who has been asking people around town if they have a sister. Yesterday, a body of a girl was found in the river, and I am investigating whether there is a connection between the two events.
[Say] Do you know anyone who might have information about this individual or the body found in the river?


> Finished chain.


> Entering new AgentExecutor chain...

[Think] I need to recall if I know anyone who could help her.
OK
[Recall] Do I know anyone who could help Becky with her case?
no information was found
[Say] Unfortunately, I don't know anyone who could help you with this case.


> Finished chain.


> Entering new AgentExecutor chain...

[Think] I should ask J

In [11]:
stop

NameError: name 'stop' is not defined

In [ ]:
input_action = FreeAction(actor="a stranger", text="pull a chair and sit next to you")
print(input_action.action_description)
output_action = scene_1.interact(input_action)
print(output_action.action_description)
print(scene_1.summary)

In [ ]:
input_action = FreeAction(actor=becky.sheet.name, text="take a sip of your coffee")
print(input_action.action_description)
output_action = scene_1.interact(input_action)
print(output_action.action_description)
print(scene_1.summary)

In [ ]:
scene_1.interact(FreeAction(becky.name, "pull a chair and sit next to you"))

In [ ]:
scene_1.say("stranger", "Hi, my name is Becky Jameson and I'm a detective. Do you have time to talk?")

In [ ]:
chat_1.say("stranger", "I'm sorry, I forgot to introduce myself. I'm Buck Jones, a dietary doctor")

In [ ]:
chat_1.summary

In [ ]:
chat_1.say("Buck Jones", "you should always remember that sugar is terrible for your health. Avoid sugar and carbs at all costs")

In [ ]:
chat_1.summary

In [ ]:
chat_1.finish_chat()

In [ ]:
chat_2 = create_chat(john, situation="You are eating cake at home", trigger="Buck Jones approached you")

In [ ]:
chat_2.say("Buck Jones", "Hi John")

In [ ]:
chat_2.say("Buck Jones", "What are you doing? Did I not warn you about that already?")